#Cats vs Dogs Classification using CNN Keras

###Loading kaggle dataset

In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 34.2MB/s]
 98% 534M/543M [00:04<00:00, 118MB/s]
100% 543M/543M [00:04<00:00, 126MB/s]
 96% 261M/271M [00:04<00:00, 82.8MB/s]
100% 271M/271M [00:04<00:00, 57.2MB/s]


In [ ]:
! unzip train.zip
! unzip test1.zip

###Importing libraries

In [ ]:
#Setting tensorflow GPU
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

###Define image properties

In [ ]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

###Preparing dataset for training model

In [ ]:
filenames = os.listdir("./train")

In [ ]:
categories=[]
for f_name in filenames:
  category = f_name.split('.')[0]
  if category == "dog":
    categories.append(1)
  else:
    categories.append(0)

df = pd.DataFrame({
    'filename' : filenames,
    'category' : categories
})

In [ ]:
df.head()

,filename,category
0,dog.761.jpg,1
1,cat.6658.jpg,0
2,cat.6100.jpg,0
3,cat.3167.jpg,0
4,dog.1784.jpg,1


###Creating the neural net model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

In [ ]:
model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

##Analyzing Model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 61, 61, 64)       2

###Defining callbacks and learning rate

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

###Managing data

In [ ]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

###Training and Validating data generator

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "./train/",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "./train/", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


###Model Training

In [ ]:
epochs=10
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
1333/1333 [==============================] - 187s 133ms/step - loss: 0.7464 - accuracy: 0.6333 - val_loss: 0.5459 - val_accuracy: 0.7345 - lr: 0.0010
Epoch 2/10
1333/1333 [==============================] - 180s 135ms/step - loss: 0.5585 - accuracy: 0.7198 - val_loss: 0.6760 - val_accuracy: 0.7542 - lr: 0.0010
Epoch 3/10
1333/1333 [==============================] - 186s 140ms/step - loss: 0.5132 - accuracy: 0.7513 - val_loss: 1.1615 - val_accuracy: 0.6745 - lr: 0.0010
Epoch 4/10
1333/1333 [==============================] - 181s 136ms/step - loss: 0.4781 - accuracy: 0.7758 - val_loss: 0.4123 - val_accuracy: 0.8164 - lr: 0.0010
Epoch 5/10
1333/1333 [==============================] - 180s 135ms/step - loss: 0.4462 - accuracy: 0.7937 - val_loss: 0.4725 - val_accuracy: 0.7982 - lr: 0.0010
Epoch 6/10
1333/1333 [==============================] - 173s 130ms/step - loss: 0.4248 - accuracy: 0.8091 - val_loss: 0.4823 - val_accuracy: 0.7710 - lr: 0.0010
Epoch 7/10
1333/1333 [============

In [ ]:
#Saving the model
model.save("model1_catsVSdogs_10epoch.h5")

###Test Data Preparation

In [ ]:
test_filenames = os.listdir("./test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:
test_df.head()

,filename
0,4183.jpg
1,10053.jpg
2,3741.jpg
3,470.jpg
4,1008.jpg


###Creating test datagenerator

In [ ]:
test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

In [ ]:
test_generator = train_datagen.flow_from_dataframe(test_df, 
                                                   "./test1/",x_col='filename',y_col=None,
                                                   target_size = Image_Size,
                                                   class_mode=None,
                                                   batch_size=batch_size)

Found 12500 validated image filenames.


###Making categorical prediction

In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


###Convert labels to categories

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)

label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

###Visualizing the prediction results

In [ ]:
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("./test1/"+filename, target_size=Image_Size)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.